In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import logging
logging.basicConfig(level=logging.INFO)

from threepy.server import WebsocketServer
from threepy.geometry import *
from threepy.materials import *
from threepy.objects import *
from threepy.visualizer import Visualizer
import time

In [ ]:
vis = Visualizer()

INFO:websockets.server:server listening on 0.0.0.0:8001
INFO:websockets.server:server listening on [::]:8001


INFO:websockets.server:connection open
INFO:root:New Websocket added. Adding to list of connections
INFO:root:Received message: {"type":"init","client":"visualizer"}
INFO:root:Connection Closed. Popping from list of connections
INFO:websockets.server:connection open
INFO:root:New Websocket added. Adding to list of connections
INFO:root:Received message: {"type":"init","client":"visualizer"}
INFO:root:Connection Closed. Popping from list of connections
INFO:websockets.server:connection open
INFO:root:New Websocket added. Adding to list of connections
INFO:root:Received message: {"type":"init","client":"visualizer"}
INFO:root:Connection Closed. Popping from list of connections
INFO:websockets.server:connection open
INFO:root:New Websocket added. Adding to list of connections
INFO:root:Received message: {"type":"init","client":"visualizer"}
INFO:root:Connection Closed. Popping from list of connections
INFO:websockets.server:connection open
INFO:root:New Websocket added. Adding to list of 

In [4]:
vis.server.is_running

True

In [5]:
await vis.set_props("box", {"position": [0, 1, 1]})

In [6]:
# Move the box in a circle
import numpy as np
import time
t = np.linspace(0, 2*np.pi, 301)
for i in range(len(t)):
    await vis.set_props("box", {"position": [np.cos(t[i]), np.sin(t[i]), 1]})
    time.sleep(0.002)

In [7]:
# Add material
mat = MeshLambertMaterial(color=0x0000ff, name="mat_blue")
await vis.add_material(mat)

In [8]:
# Add a sphere geometry
sphere = SphereGeometry(radius=0.5, name="sphere")
await vis.add_geometry(sphere)

In [9]:
# Add mesh
blue_sphere = SimpleMeshRef(geometry_name=sphere.name, material_name=mat.name, name="blue_sphere")
await vis.add_object(blue_sphere)

In [36]:
# Add mesh to scene and move
await vis.add_child("scene", "blue_sphere")
await vis.set_props("blue_sphere", {"position": [0, 0, 1]})

In [52]:
# Add cone with new geometry and material
green_cone = SimpleMesh(geometry=ConeGeometry(), material=MeshLambertMaterial(color=0x00ff00), name="green_cone")
await vis.add_object(green_cone, parent_name="scene")

In [70]:
import numpy as np
t = np.linspace(0, 2*np.pi, 31)
pos = np.zeros((len(t), 3))
pos[:, 0] = np.cos(t)
pos[:, 1] = np.sin(t)
pos[:, 2] = 1.0 + t / (2*np.pi) * 4

In [73]:
from threepy.animations import *
cone_kf = KeyframeTrack("green_cone.position[y]", [0, 1], [0.0, 10.0], Interpolation.InterpolateSmooth)
box_kf = KeyframeTrack("box.position", list(t / (2 * np.pi) * 3), list(pos.flatten()), Interpolation.InterpolateSmooth, value_type='vector')
clip = AnimationClip("animation_cone", -1, [cone_kf, box_kf])
await vis.add_animation(clip)

In [74]:
await vis.load_animation(clip.name)

In [21]:
clip.lower()

{'uuid': '15ba63be-1531-491f-9828-66784ac2a024',
 'name': 'cone',
 'duration': -1,
 'blendMode': 2500,
 'tracks': [{'name': 'green_cone.position[x]',
   'type': 'number',
   'times': [0, 1],
   'values': [0.0, 10.0],
   'interpolation': 2302}]}

In [8]:
vis.server.stop()